### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [5]:
#calculate the total # of unique players by their SN
total_players=purchase_data["SN"].nunique()
print("Total Players:", total_players)

Total Players: 576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [7]:
#calculate number of unique items available, average price, total number of unique purchases, and total revenue
number_unique_items=purchase_data["Item ID"].nunique()
average_price=purchase_data["Price"].mean()
number_of_purchases=purchase_data["Purchase ID"].nunique()
total_revenue=purchase_data["Price"].sum()

#make data above into data form that can be applied to a dataframe
data={"Number of Unique Items": [number_unique_items],"Average Price": [average_price],"Number of Purchases":[number_of_purchases],"Total Revenue":[total_revenue]}

#make dataframe with data
summary_table=pd.DataFrame(data=data)

#format dataframe
summary_table["Average Price"] = summary_table["Average Price"].map("${:.2f}".format)
summary_table["Total Revenue"] = summary_table["Total Revenue"].map("${:,.2f}".format)
summary_table

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [8]:
#aggregate data to remove multiple purchases by the same person
purchase_data_by_SN= purchase_data.groupby(["Gender","SN"]).agg({"Purchase ID":"count","Price":
                                                                     ["mean","sum"],"Age":"mean"}).reset_index()
purchase_data_by_SN.columns=["Gender","SN","# of Purchases","Mean Price","Sum of Price","Age"]
purchase_data_by_SN.head()

,Gender,SN,# of Purchases,Mean Price,Sum of Price,Age
0,Female,Adastirin33,1,4.48,4.48,35
1,Female,Aerithllora36,2,4.32,8.64,29
2,Female,Aethedru70,1,3.54,3.54,21
3,Female,Aidain51,1,3.45,3.45,39
4,Female,Aiduesu86,1,4.48,4.48,22


In [10]:
#group data above by gender to see how many unique players are in each gender
gender_breakdown=purchase_data_by_SN.groupby(["Gender"]).agg({"Gender":"count"})
del gender_breakdown.index.name
gender_breakdown.columns=["Total Count"]
gender_breakdown["% of Players"]=gender_breakdown["Total Count"]/gender_breakdown["Total Count"].sum()*100
gender_breakdown["% of Players"]=gender_breakdown["% of Players"].map("{:,.1f}%".format)
gender_breakdown

,Total Count,% of Players
Female,81,14.1%
Male,484,84.0%
Other / Non-Disclosed,11,1.9%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [13]:
#group original data by gender to see individual transaction data by gender (will be merged later)
gender_stats=purchase_data.groupby("Gender").agg({"Purchase ID":"count","Price":["mean","sum"]})
gender_stats.columns = gender_stats.columns.droplevel(0)
gender_stats.rename(inplace=True,
                    columns={"count":"Purchase Count",
                             "mean":"Average Purchase Price",
                            "sum":"Total Purchase Value"})
gender_stats

,Purchase Count,Average Purchase Price,Total Purchase Value
Gender,,,
Female,113,3.203009,361.94
Male,652,3.017853,1967.64
Other / Non-Disclosed,15,3.346000,50.19


In [15]:
#group original data by gender and SN to see transaction data at the player level (to see how many players in each gender)
SN_agg=purchase_data.groupby(["SN","Gender"])[["Price"]].sum()
SN_Gender_agg=SN_agg.groupby(["Gender"]).mean()
SN_Gender_agg

,Price
Gender,
Female,4.468395
Male,4.065372
Other / Non-Disclosed,4.562727


In [17]:
#merge tables above and format
merged_table = pd.merge(gender_stats, SN_Gender_agg, on="Gender", how='inner')
merged_table.rename(inplace=True,
                    columns={"Price":"Avg Total Purchase per Person"}) 
merged_table["Average Purchase Price"]=merged_table["Average Purchase Price"].map("${:,.2f}".format)
merged_table["Total Purchase Value"]=merged_table["Total Purchase Value"].map("${:,.2f}".format)
merged_table["Avg Total Purchase per Person"]=merged_table["Avg Total Purchase per Person"].map("${:,.2f}".format)
merged_table

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [18]:
# make bins
bins = [0, 9, 14, 19, 24, 29,34,39,100]

# make bin names
group_names = ["<10", "10-14", "15-19", "20-24", "25-29","30-34","35-39","40+"]

In [19]:
#group age data according to bins
purchase_data_by_SN["Groups"] = pd.cut(purchase_data_by_SN["Age"], bins, labels=group_names)
purchase_data_by_SN.head()

,Gender,SN,# of Purchases,Mean Price,Sum of Price,Age,Groups
0,Female,Adastirin33,1,4.48,4.48,35,35-39
1,Female,Aerithllora36,2,4.32,8.64,29,25-29
2,Female,Aethedru70,1,3.54,3.54,21,20-24
3,Female,Aidain51,1,3.45,3.45,39,35-39
4,Female,Aiduesu86,1,4.48,4.48,22,20-24


In [20]:
#group by age groups to determine how many players lie within each
purchase_data_age_demographics=purchase_data_by_SN.groupby(["Groups"])[["Age"]].count()

purchase_data_age_demographics.rename(columns={"Age":"Number of Players"},inplace=True)

purchase_data_age_demographics["% of players"]=purchase_data_age_demographics["Number of Players"]/sum(purchase_data_age_demographics["Number of Players"])*100
purchase_data_age_demographics["% of players"] = purchase_data_age_demographics["% of players"].map("{:.2f}%".format)
del purchase_data_age_demographics.index.name
purchase_data_age_demographics

,Number of Players,% of players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [21]:
#aggragate original data to see transaction data at the highest level (to calculate avegage price per age group)
purchase_data["Groups"] = pd.cut(purchase_data["Age"], bins, labels=group_names)
avg_purchase_per_group=purchase_data.groupby(["Groups"]) [["Price"]].mean()
avg_purchase_per_group.rename(columns={"Price":"Average Purchase Price"},inplace=True)
avg_purchase_per_group["Average Purchase Price"] = avg_purchase_per_group["Average Purchase Price"].map("${:.2f}".format)

avg_purchase_per_group

,Average Purchase Price
Groups,
<10,$3.35
10-14,$2.96
15-19,$3.04
20-24,$3.05
25-29,$2.90
30-34,$2.93
35-39,$3.60
40+,$2.94


In [22]:
#aggragate data at the player level to calculate purchases/person in each age group
age_purchasing_analysis=purchase_data_by_SN.groupby(["Groups"]).agg({"# of Purchases":"sum","Sum of Price":"sum"}) 
age_purchasing_analysis.columns=["Purchase Count","Total Purchase Value"]
age_purchasing_analysis["Average Total Purchases/Person"]=age_purchasing_analysis["Total Purchase Value"]/purchase_data_age_demographics["Number of Players"]
age_purchasing_analysis["Total Purchase Value"] = age_purchasing_analysis["Total Purchase Value"].map("${:.2f}".format)
age_purchasing_analysis["Average Total Purchases/Person"] = age_purchasing_analysis["Average Total Purchases/Person"].map("${:.2f}".format)

age_purchasing_analysis

,Purchase Count,Total Purchase Value,Average Total Purchases/Person
Groups,,,
<10,23,$77.13,$4.54
10-14,28,$82.78,$3.76
15-19,136,$412.89,$3.86
20-24,365,$1114.06,$4.32
25-29,101,$293.00,$3.81
30-34,73,$214.00,$4.12
35-39,41,$147.67,$4.76
40+,13,$38.24,$3.19


In [24]:
#merge tables above
age_purchasing_analysis_merged=pd.merge(age_purchasing_analysis, avg_purchase_per_group, on="Groups", how='right')
age_purchasing_analysis_merged
age_purchasing_analysis_merged=age_purchasing_analysis_merged[["Purchase Count","Average Purchase Price","Total Purchase Value","Average Total Purchases/Person"]]

age_purchasing_analysis_merged

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchases/Person
Groups,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,$1114.06,$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [25]:
#sort SN data by Sum of Revenues
purchase_data_by_SN_sorted=purchase_data_by_SN.sort_values(by="Sum of Price",ascending=False).reset_index()
purchase_data_by_SN_sorted.head()

,index,Gender,SN,# of Purchases,Mean Price,Sum of Price,Age,Groups
0,390,Male,Lisosia93,5,3.792000,18.96,25,25-29
1,289,Male,Idastidru52,4,3.862500,15.45,24,20-24
2,14,Female,Chamjask73,3,4.610000,13.83,22,20-24
3,315,Male,Iral74,4,3.405000,13.62,21,20-24
4,321,Male,Iskadarya95,3,4.366667,13.10,20,20-24


In [36]:
#extract columns of interest
top_spenders=purchase_data_by_SN_sorted[["SN","# of Purchases","Mean Price","Sum of Price"]]
top_spenders["Sum of Price"] = top_spenders["Sum of Price"].map("{:.2f}".format)
top_spenders["Mean Price"] = top_spenders["Mean Price"].map("${:.2f}".format)
top_spenders.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,SN,# of Purchases,Mean Price,Sum of Price
0,Lisosia93,5,$3.79,18.96
1,Idastidru52,4,$3.86,15.45
2,Chamjask73,3,$4.61,13.83
3,Iral74,4,$3.40,13.62
4,Iskadarya95,3,$4.37,13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [37]:
#testing why Fuinal Critic item name yields most purchases but not when sorted by Item ID-->because corresponds to multiple item IDs
purchase_data.loc[purchase_data["Item Name"]=="Final Critic"]

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Groups
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
99,99,Haisrisuir60,23,Male,101,Final Critic,4.19,20-24
252,252,Tyaelo67,39,Male,92,Final Critic,4.88,35-39
273,273,Phyali88,15,Female,92,Final Critic,4.88,15-19
277,277,Ennalmol65,24,Male,92,Final Critic,4.88,20-24
411,411,Lisico81,10,Male,101,Final Critic,4.19,10-14
536,536,Siallylis44,20,Male,101,Final Critic,4.19,20-24
712,712,Lisilsa62,25,Male,92,Final Critic,4.88,25-29
722,722,Ilarin91,22,Male,92,Final Critic,4.88,20-24
767,767,Ilmol66,8,Female,92,Final Critic,4.88,<10


In [38]:
#create most popular items output without formatting so that I can manipulate number later without having turned them into strings
most_popular_items=purchase_data.groupby(["Item ID","Item Name"]).agg({"SN":"count","Price":"sum"})
most_popular_items["Item Price"]=most_popular_items["Price"]/most_popular_items["SN"]
most_popular_items.sort_values(by="SN",ascending=False,inplace=True)
most_popular_items.columns=["Purchase Count","Total Purchase Value","Item Price"]
most_popular_items.head()

,,Purchase Count,Total Purchase Value,Item Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,50.76,4.23
145,Fiery Glass Crusader,9,41.22,4.58
108,"Extraction, Quickblade Of Trembling Hands",9,31.77,3.53
82,Nirvana,9,44.10,4.90
19,"Pursuit, Cudgel of Necromancy",8,8.16,1.02


In [39]:
#create most popular items output with formatting
most_popular_items_formatted=most_popular_items[['Purchase Count', 'Total Purchase Value', 'Item Price']]
most_popular_items_formatted["Total Purchase Value"] = most_popular_items_formatted["Total Purchase Value"].map("${:.2f}".format)
most_popular_items_formatted["Item Price"] = most_popular_items_formatted["Item Price"].map("${:.2f}".format)
most_popular_items_formatted.columns=["Purchase Count","Total Purchase Value","Item Price"]
most_popular_items_formatted.head()

,,Purchase Count,Total Purchase Value,Item Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
145,Fiery Glass Crusader,9,$41.22,$4.58
108,"Extraction, Quickblade Of Trembling Hands",9,$31.77,$3.53
82,Nirvana,9,$44.10,$4.90
19,"Pursuit, Cudgel of Necromancy",8,$8.16,$1.02


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [40]:
#create most popular items output with formatting
most_profitable_items=most_popular_items.sort_values(by="Total Purchase Value",ascending=False)
most_profitable_items["Total Purchase Value"] = most_profitable_items["Total Purchase Value"].map("${:.2f}".format)
most_profitable_items["Item Price"] = most_profitable_items["Item Price"].map("${:.2f}".format)
most_profitable_items.head()

,,Purchase Count,Total Purchase Value,Item Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
82,Nirvana,9,$44.10,$4.90
145,Fiery Glass Crusader,9,$41.22,$4.58
92,Final Critic,8,$39.04,$4.88
103,Singed Scalpel,8,$34.80,$4.35
